# Tracer Advection Stencil

## Start a parallel cluster

In [ ]:
import ipyparallel as ipp

cluster = ipp.Cluster(engines='mpi', n=6).start_and_connect_sync()
%autopx


## Input parameters

In [ ]:
%%capture
from mpi4py import MPI
import copy as cp
import functions as func
import numpy as np
import importlib

importlib.reload(func)

mpi_comm = MPI.COMM_WORLD

backend = 'numpy'
layout = (1, 1)
nx, ny, nz, nhalo = 101, 101, 80, 3

dt_acoustic = 300.
advection_timestep = 3600.
nSteps_advection = 10
print_advectionProgress = True
plot_output = True
figure_everyNsteps = 50

if figure_everyNsteps > nSteps_advection:
    figure_everyNsteps = nSteps_advection

tracer_archive_frequency = figure_everyNsteps

grid_type = 0
hord = 6

pressure_base = 10
smoke_base = 1
smoke_center_tile = 0
density = 1

## Prepare state for advection

In [ ]:
%%capture
dimensions, units, origins, rank = func.define_dimensionsUnitsOrigins(nx, ny, nz, nhalo, mpi_comm)
smoke_dict = {'center_tile': smoke_center_tile, 'rank': rank, 'smoke_base': smoke_base}
pressure_dict = {'pressure_base': pressure_base}
fvt_dict = {'grid_type': grid_type, 'hord': hord}

configuration = func.configure_domain(layout, mpi_comm, dimensions, backend=backend)
initialState = func.create_initialState_testCase1(configuration['grid_data'], dimensions, units, origins, backend, smoke_dict, pressure_dict)
tracAdv_data, tracAdv = func.prepare_everythingForAdvection(configuration, initialState['uC'], initialState['vC'], initialState['delp'], {'smoke': initialState['smoke']}, density, dimensions, units, origins, backend, fvt_dict, dt_acoustic=dt_acoustic)
tracAdv_dataInit = cp.deepcopy(tracAdv_data)

## Advection loop

In [ ]:
%%capture
tracer_archive = [cp.deepcopy(tracAdv_data['tracers']['smoke'])]
timestep_archive = [0]
for step in range(nSteps_advection):    
    if (rank == 0) and (step+1)%100 == 0 and print_advectionProgress:
        print('Step: %d' % (step+1))
    #tracAdv_data = func.run_advectionStepWithReset(tracAdv_dataInit, tracAdv_data, tracAdv, advection_timestep, mpi_rank=rank)
    tracAdv_data = func.run_advectionStepWithReset(tracAdv_dataInit, tracAdv_data, tracAdv, advection_timestep)

    if (step+1)%tracer_archive_frequency == 0:
        tracer_archive.append(cp.deepcopy(tracAdv_data['tracers']['smoke']))
        timestep_archive.append(step+1)

In [ ]:
#%%capture
if plot_output:
    lon, lat = func.get_lonLat_agrid(configuration['grid_data'], dimensions, units, origins, backend)
    lon_global = configuration['communicator'].gather(lon)
    lat_global = configuration['communicator'].gather(lat)
    smoke_global = []
    for step in range(len(timestep_archive)):
        smoke_global.append(configuration['communicator'].gather(tracer_archive[step]))

    saveDir = '/home/ajdas/pace/advection_test/figs/'
    fOut = saveDir + 'advection_step_'

    if rank == 0:
        smoke_global = np.squeeze(np.stack(smoke_global))
        for plotStep in range(len(timestep_archive)):
            print('Plotting time step %s / %s' % (timestep_archive[plotStep], nSteps_advection))
            func.plot_projection_field(lon_global.data, lat_global.data, smoke_global[plotStep], 
                                       cmap='viridis', vmin=0, vmax=1, units='kg', 
                                       title='Smoke after %s advection steps' % timestep_archive[plotStep], 
                                       fSave=fOut+'{:03d}'.format(timestep_archive[plotStep]) + '.png')

In [ ]:
%autopx

In [ ]:
cluster.shutdown()